In [1]:
#installing the jupyter notebook dependencies for gmaps
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
%matplotlib notebook

In [3]:
# Dependencies
from config import gkey
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import gmaps
# Configure gmaps
gmaps.configure(api_key=gkey)
from us import states


In [4]:
#Preparing the sales data frame
csv_file = "./Data/Sales.csv"
sales_df = pd.read_csv(csv_file)
sales_df.set_index(['Record_ID'])
sales_df.dropna()
sales_df.head()

C:\Users\mghan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Record_ID,Date_of_Payment,Recipient_Zip_Code,Total_Amount_of_Payment_USDollars,Teaching_Hospital_ID,Teaching_Hospital_Name
0,601890005,04/24/2018,52242,35708.00,6639.0,UNIVERSITY OF IOWA HOSP & CLINICS
1,601982005,10/08/2018,84132,2315.00,7144.0,U OF U HOSPITALS & CLINICS
2,573426201,02/02/2018,97225,465.00,7183.0,PROVIDENCE ST. VINCENT MEDICAL CTR
3,613716105,01/17/2018,65212,1318.83,6652.0,UNIV OF MISSOURI HEALTH CARE
4,576948161,08/30/2018,20007,52.50,6745.0,GEORGETOWN UNIVERSITY HOSPITAL


In [9]:
# Calculating the Total sales by per zip code(using Pivot table)
pivot_sales_zipcode = pd.pivot_table(sales_df,values='Total_Amount_of_Payment_USDollars', index=['Recipient_Zip_Code'] ,aggfunc=np.sum).reset_index()

#Getting the top ranking zipcode oedered by totalsales
top_Sales_zipcode =pivot_sales_zipcode.sort_values(by ='Total_Amount_of_Payment_USDollars',ascending = False).head(1000)
top_Sales_zipcode

,Recipient_Zip_Code,Total_Amount_of_Payment_USDollars
3106,27704-2128,2.082956e+08
9292,77030,1.874129e+08
198,02115,1.021911e+08
268,02215,9.610630e+07
2013,19104,6.403545e+07
1109,10065,5.230670e+07
11998,94143-3010,5.151764e+07
8030,63110,4.443377e+07
185,02114,4.234696e+07
11990,94143,4.072098e+07


In [10]:
#Getting the latitude and longitude from the top zipcodes
#Pip install pgeocode is done
import pgeocode
lats=[]
lons =[]
nomi = pgeocode.Nominatim('us')
for zips in top_Sales_zipcode['Recipient_Zip_Code']:
    lat = nomi.query_postal_code(zips).latitude
    lon = nomi.query_postal_code(zips).longitude
    lats.append(lat)
    lons.append(lon)
    
lats    
top_Sales_zipcode['lats']= lats
top_Sales_zipcode['lons']= lons
top_Sales_zipcode = top_Sales_zipcode[top_Sales_zipcode['lats'].notnull()]
top_Sales_zipcode

,Recipient_Zip_Code,Total_Amount_of_Payment_USDollars,lats,lons
9292,77030,1.874129e+08,29.7041,-95.4010
198,02115,1.021911e+08,42.3427,-71.0922
268,02215,9.610630e+07,42.3471,-71.1027
2013,19104,6.403545e+07,39.9597,-75.2024
1109,10065,5.230670e+07,40.7651,-73.9638
8030,63110,4.443377e+07,38.6185,-90.2564
185,02114,4.234696e+07,42.3611,-71.0682
11990,94143,4.072098e+07,37.7631,-122.4586
6242,44195,3.472694e+07,41.6857,-81.6728
10924,90095,3.362551e+07,33.7866,-118.2987


In [11]:
# Storing 'Lat' and 'Lng' into  locations 
locations = top_Sales_zipcode [['lats', 'lons']].astype(float)
locations
# Converting total sales to float and store
total_sales = top_Sales_zipcode ['Total_Amount_of_Payment_USDollars'].astype(float)
total_sales
locations

,lats,lons
9292,29.7041,-95.4010
198,42.3427,-71.0922
268,42.3471,-71.1027
2013,39.9597,-75.2024
1109,40.7651,-73.9638
8030,38.6185,-90.2564
185,42.3611,-71.0682
11990,37.7631,-122.4586
6242,41.6857,-81.6728
10924,33.7866,-118.2987


In [13]:
# Customizing the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [14]:
# Assigning the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Adding the layer to the map
fig.add_layer(markers)


In [15]:
#configuring the google heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=total_sales,dissipating=True,
                                )
                                 

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
#configuring the google map with zipcode marker and their total sales details
fig = gmaps.figure()

sales_layer = gmaps.symbol_layer(locations,fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Total Sales amount: ${sale}" for sale in total_sales]
)


fig = gmaps.figure()
fig.add_layer(sales_layer)

fig

Figure(layout=FigureLayout(height='420px'))